In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
from scipy.interpolate import interp1d

In [2]:
def cutSegment(size,data,segmt):   
    segdata=pd.DataFrame()
    for i in range(len(segmt)-1):
        length=segmt[i+1]-segmt[i]
        part=pd.DataFrame()       
        if length > size:
            n = length - size
            part=data.iloc[segmt[i]:(segmt[i+1]),:]
            part=part.reset_index(drop=True)
            part.drop(part.tail(n).index,inplace=True) 
            segdata=segdata.append(part,ignore_index=True)
    return segdata

In [4]:
# read xlsx file with # time, current, soc and vol1-198
wb = pd.read_excel(
    "../../data/filt-data/HMZABAAH9MF014494_412_512.xlsx", sheet_name="HMZABAAH9MF014494_412_512",
)  
# store data to data frame
df = pd.DataFrame(wb)
df



,time,current,BMSSOC,VCU_BMSChargeState,ocv,CellVoltage_1,CellVoltage_2,CellVoltage_3,CellVoltage_4,CellVoltage_5,...,CellVoltage_189,CellVoltage_190,CellVoltage_191,CellVoltage_192,CellVoltage_193,CellVoltage_194,CellVoltage_195,CellVoltage_196,CellVoltage_197,CellVoltage_198
0,2022-05-05 21:35:10,0.100,93.5,3,3349,3349,3349,3349,3349,3348,...,3349,3349,3348,3348,3348,3348,3349,3349,NaN,NaN
1,2022-04-12 01:40:38,-105.299,80.0,3,3421,3421,3420,3421,3420,3419,...,3422,3419,3420,3418,3420,3421,3420,3420,NaN,NaN
2,2022-04-12 01:40:43,-105.299,80.0,3,3421,3421,3420,3421,3420,3419,...,3422,3419,3420,3418,3420,3421,3420,3420,NaN,NaN
3,2022-04-12 01:40:48,-105.299,80.0,3,3421,3421,3420,3421,3421,3419,...,3422,3419,3420,3419,3421,3421,3421,3420,NaN,NaN
4,2022-04-12 01:40:53,-105.299,80.5,3,3422,3421,3420,3421,3421,3419,...,3422,3419,3420,3419,3421,3421,3421,3420,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6471,2022-05-05 21:34:50,0.100,93.5,3,3350,3350,3349,3350,3350,3349,...,3349,3349,3349,3349,3349,3349,3350,3349,NaN,NaN
6472,2022-05-05 21:34:55,0.100,93.5,3,3350,3350,3349,3350,3349,3349,...,3349,3349,3348,3349,3349,3349,3349,3349,NaN,NaN
6473,2022-05-05 21:35:00,0.100,93.5,3,3350,3349,3349,3349,3349,3348,...,3349,3349,3348,3349,3349,3349,3349,3349,NaN,NaN
6474,2022-05-05 21:35:05,0.100,93.5,3,3350,3349,3349,3349,3349,3348,...,3349,3349,3348,3348,3348,3349,3349,3349,NaN,NaN


In [11]:
## data processing
# create new data frame to store processed data
centerdf=pd.DataFrame()
# store time, current and soc to data frame
n = 3
centerdf["time"] = df.iloc[:, 0]
centerdf["current"] = df.iloc[:, 1]
centerdf["soc"] = df.iloc[:,2]
centerdf["ocv"] = df.iloc[:,4]

# store difference between mean of cell voltage for each cell into data frame
for i in range((int(df.shape[1] - 5))):

    centerdf["vol" + str(i + 1)] = df.iloc[:, i + 5] 

# centerdf = centerdf.dropna(axis=0, how='any')
# visualize
centerdf

,time,current,soc,ocv,vol1,vol2,vol3,vol4,vol5,vol6,...,vol189,vol190,vol191,vol192,vol193,vol194,vol195,vol196,vol197,vol198
0,2022-05-05 21:35:10,0.100,93.5,3349,3349,3349,3349,3349,3348,3349,...,3349,3349,3348,3348,3348,3348,3349,3349,NaN,NaN
1,2022-04-12 01:40:38,-105.299,80.0,3421,3421,3420,3421,3420,3419,3420,...,3422,3419,3420,3418,3420,3421,3420,3420,NaN,NaN
2,2022-04-12 01:40:43,-105.299,80.0,3421,3421,3420,3421,3420,3419,3420,...,3422,3419,3420,3418,3420,3421,3420,3420,NaN,NaN
3,2022-04-12 01:40:48,-105.299,80.0,3421,3421,3420,3421,3421,3419,3420,...,3422,3419,3420,3419,3421,3421,3421,3420,NaN,NaN
4,2022-04-12 01:40:53,-105.299,80.5,3422,3421,3420,3421,3421,3419,3420,...,3422,3419,3420,3419,3421,3421,3421,3420,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6471,2022-05-05 21:34:50,0.100,93.5,3350,3350,3349,3350,3350,3349,3350,...,3349,3349,3349,3349,3349,3349,3350,3349,NaN,NaN
6472,2022-05-05 21:34:55,0.100,93.5,3350,3350,3349,3350,3349,3349,3350,...,3349,3349,3348,3349,3349,3349,3349,3349,NaN,NaN
6473,2022-05-05 21:35:00,0.100,93.5,3350,3349,3349,3349,3349,3348,3349,...,3349,3349,3348,3349,3349,3349,3349,3349,NaN,NaN
6474,2022-05-05 21:35:05,0.100,93.5,3350,3349,3349,3349,3349,3348,3349,...,3349,3349,3348,3348,3348,3349,3349,3349,NaN,NaN


In [37]:
# relax-v
# charge_current = centerdf["current"]
# timestamp = centerdf["time"]
# vol = centerdf["vol"]
# charge_current_list = []
# timestamp_list = []
# vol_list = []
# for i in range(len(charge_current)):
#     # eliminate error code
#     if i != 10070 and i !=10074 and i!= 10078 and i!=10082 and i!=22350:
#         if charge_current[i] > -10 and charge_current[i] < 10:
#             charge_current_list.append(charge_current[i])
#             timestamp_list.append(datetime.strptime(timestamp[i], "%Y-%m-%d %H:%M:%S"))
#             vol_list.append(vol[i])
# print(len(charge_current_list))
# print(len(timestamp_list))

    

    

KeyError: 'vol'

In [12]:
# separate data by time
seg=[0]
for i in range(0,(int(df.shape[0])-1)):
    if df.iloc[i+1,3] == 3:
        if datetime.strptime(df.iloc[i+1,0], "%Y-%m-%d %H:%M:%S")-datetime.strptime(df.iloc[i,0], "%Y-%m-%d %H:%M:%S")>=timedelta(seconds=300):
            seg.append(i+1)
seg.append(len(df))
print(seg)
# insert first value as default to fill the segment
segdf=cutSegment(370,centerdf,seg)
segdf


            
# if NaN print True

[0, 257, 489, 749, 975, 1368, 1757, 2003, 2337, 2438, 2682, 2927, 3109, 3354, 3609, 3847, 4098, 4449, 4697, 4956, 5195, 5406, 5631, 5863, 6109, 6292, 6476]


,time,current,soc,ocv,vol1,vol2,vol3,vol4,vol5,vol6,...,vol189,vol190,vol191,vol192,vol193,vol194,vol195,vol196,vol197,vol198
0,2022-04-17 12:55:01,-53.799,80.0,3392,3392,3392,3392,3392,3391,3392,...,3394,3392,3392,3392,3392,3393,3393,3393,NaN,NaN
1,2022-04-17 12:55:06,-53.799,80.0,3393,3392,3392,3392,3392,3392,3392,...,3393,3392,3392,3392,3392,3393,3393,3393,NaN,NaN
2,2022-04-17 12:55:11,-53.799,80.0,3392,3392,3392,3392,3392,3392,3392,...,3394,3392,3392,3392,3392,3393,3393,3393,NaN,NaN
3,2022-04-17 12:55:16,-53.799,80.0,3392,3392,3392,3392,3392,3392,3392,...,3393,3392,3392,3392,3392,3393,3393,3393,NaN,NaN
4,2022-04-17 12:55:21,-53.799,80.0,3393,3392,3392,3392,3392,3392,3392,...,3394,3392,3392,3392,3392,3393,3393,3393,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,2022-04-20 09:12:22,-43.399,99.0,3557,3560,3561,3549,3559,3450,3551,...,3530,3549,3548,3558,3562,3542,3547,3545,NaN,NaN
736,2022-04-20 09:12:27,-43.399,99.0,3570,3572,3573,3560,3572,3451,3563,...,3539,3562,3561,3574,3576,3553,3559,3556,NaN,NaN
737,2022-04-20 09:12:32,-43.399,99.0,3583,3586,3588,3573,3586,3452,3576,...,3550,3576,3576,3590,3593,3566,3572,3569,NaN,NaN
738,2022-04-20 09:12:37,-15.599,99.0,3572,3574,3576,3561,3575,3439,3565,...,3537,3565,3565,3580,3582,3553,3561,3557,NaN,NaN


In [10]:
segdf

""


In [13]:
# save processed data into csv format for further training
vol=[]
cur=[]
vol_num=[]
soc = []
ocv = []

for i in range(198):
    vol=vol+segdf.iloc[:,i+3].values.tolist()
    cur=cur+segdf.iloc[:,1].values.tolist()
    soc=soc+segdf.iloc[:,2].values.tolist()
    ocv = ocv + segdf.iloc[:,3].values.tolist()
    # for estimate state-vol
    # soc = soc+[100]
    # cur = cur+[0]
    # vol = vol+[segdf.iloc[:,i+3].values.tolist()[-1]-200]

tsdf=pd.DataFrame()
tsdf["current"]=np.float64(cur)
tsdf["vol"]=np.float64(vol)
tsdf["soc"] = np.float64(soc)
tsdf["ocv"] = np.float64(ocv)
# tsdf.to_csv("car1-soc.csv",index=False)
tsdf.to_csv("../../data/train-data/ocv_HMZABAAH9MF014494_412_512.csv",mode="w",index=False,header=["current","vol","soc","ocv"],na_rep='nan')

In [9]:
tsdf

,current,vol,soc
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
1471927,-19.399,28.282828,99.5
1471928,-19.399,29.398990,99.5
1471929,-12.799,29.500000,100.0
1471930,0.100,26.292929,100.0


In [ ]:
import numpy as np


def MinMaxScaler(data):
    """Min Max normalizer.

    Args:
      - data: original data

    Returns:
      - norm_data: normalized data
    """
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    norm_data = numerator / (denominator + 1e-7)
    return norm_data


def sine_data_generation(no, seq_len, dim):
    """Sine data generation.

    Args:
      - no: the number of samples
      - seq_len: sequence length of the time-series
      - dim: feature dimensions

    Returns:
      - data: generated data
    """
    # Initialize the output
    data = list()

    # Generate sine data
    for i in range(no):
        # Initialize each time-series
        temp = list()
        # For each feature
        for k in range(dim):
            # Randomly drawn frequency and phase
            freq = np.random.uniform(0, 0.1)
            phase = np.random.uniform(0, 0.1)

            # Generate sine signal based on the drawn frequency and phase
            temp_data = [np.sin(freq * j + phase) for j in range(seq_len)]
            temp.append(temp_data)

        # Align row/column
        temp = np.transpose(np.asarray(temp))
        # Normalize to [0,1]
        temp = (temp + 1) * 0.5
        # Stack the generated data
        data.append(temp)

    return data


def real_data_loading(data_name, seq_len):
    """Load and preprocess real-world datasets.

    Args:
      - data_name: stock or energy
      - seq_len: sequence length

    Returns:
      - data: preprocessed data.
    """

    ori_data = np.loadtxt( str(data_name) + ".csv", delimiter=",", skiprows=1)

    # Flip the data to make chronological data
    # ori_data = ori_data[::-1]
    # Normalize the data
    ori_data = MinMaxScaler(ori_data)
    # Preprocess the dataset
    temp_data = []
    time = []
    # Cut data by sequence length
    for i in range(0, int(len(ori_data) / seq_len)):
        _x = ori_data[i * seq_len : i * seq_len + seq_len]
        temp_data.append(_x)

    # Mix the datasets (to make it similar to i.i.d)
    idx = np.random.permutation(len(temp_data))
    data = []
    for i in range(len(temp_data)):
        data.append(temp_data[idx[i]])
        time.append(seq_len)
    data = np.array(data)
    time = np.array(time)
    return data, time

In [ ]:
test_data = real_data_loading("train-data/car1-soc",370)
test_data[0].shape

In [ ]:
ori_data,T = real_data_loading("oridata-soc",370)
pic_num = 16
index = 1
index = index*pic_num
# for i in range(1,pic_num+1):
#     plt.subplot(4,4,i)
#     plt.plot(ori_data[i+index,:,1])

plt.figure()
for i in range(1,pic_num+1):
    plt.subplot(4,4,i)
    plt.plot(ori_data[i+index,:,2])